In [1]:
import pandas as pd 

df = pd.read_csv('(2022-09-08) Transport_all.csv', dtype={'WorkerID': 'str', 'SectorID': 'str'})
df['EventInstantLocalDTS'] = pd.to_datetime(df['EventInstantLocalDTS'], infer_datetime_format=True)  

### Bed Floor Mapping

In [2]:
#load bed map and grouping map 
df_bed_map = pd.read_excel('BedMap2020 - from Cecilia.xlsx', dtype={'BedID':'str'})
df_grouping_map = pd.read_excel('2022 Proximity Grouping Transport.xlsx')
df_grouping_map = df_grouping_map.rename(columns = {"EPIC Name/Care area/Department name" : "Care area/Department name"})
bed_map = dict(zip(df_bed_map[~df_bed_map.RecordNM.isna()].RecordNM, df_bed_map[~df_bed_map.RecordNM.isna()].DepartmentDSC))

#add missing values in grouping map 
df_grouping_map = df_grouping_map.append({'Care area/Department name':'Blake 4 Endo Pool Room','Floor':'Blake 4'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'ED ACUTE VIRTUAL','Floor':'Lunder 1'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'ED FT Virtual','Floor':'Lunder 1'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'ED InOut','Floor':'Lunder 1'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'ED SUPER TRACK VIRTUAL','Floor':'Lunder 1'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'ED SUPER TRACK VIRTUAL','Floor':'Lunder 1'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'ED Urgent Arrival','Floor':'Ellison 1'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'ED Urgent Virtual','Floor':'Ellison 1'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH CC MEL HMONC YAW7','Floor':'Yawkey 7'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH GI SOC SVC YAWY9A','Floor':'Yawkey 9'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH IMG CT OP YAW3','Floor':'Yawkey 3'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH IMG NM ENDOCR WAC7','Floor':'Wang 7'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH IMG PET MG WHT2','Floor':'White 2'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH IMG US OP YAW6','Floor':'Yawkey 6'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH IMG XR OP WAC2','Floor':'Wang 2'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH MEEI Oculer Plastics 01','Floor':'MEEI 1'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH PALL CARE YAW7E','Floor':'Yawkey 7'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH PED CARD YAW6','Floor':'Yawkey 6'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH PED HEMONC YAW8','Floor':'Yawkey 8'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH PED HEMONC YAW8','Floor':'Yawkey 8'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name':'MGH PSYCH WACC815','Floor':'Wang 8'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name': 'MGH RAI SCAN LAB WAC7','Floor':'Wang 7'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name': 'MGP IV RADIOLOGY BLK2','Floor':'Blake 7'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name': 'MGP PED SURG WAR','Floor':'Warren 11'} 
                    ,ignore_index=True)
df_grouping_map = df_grouping_map.append({'Care area/Department name': 'MGP VINC IVF YAW10','Floor':'Yawkey 10'} 
                    ,ignore_index=True)
df_grouping_map.loc[df_grouping_map['Care area/Department name'] == 'MGH IMG US WRN9', 'Floor'] = 'Warren 9'

#keep procedural rooms as floor name  
df_grouping_map.loc[df_grouping_map['Category'].str.startswith('Procedural:', na = False), 'Floor'] = df_grouping_map['Category']
df_grouping_map.loc[df_grouping_map['Category'] == 'Procedural: EP', 'Floor'] = df_grouping_map['Care area/Department name']
df_grouping_map.loc[df_grouping_map['Category'] == 'Procedural: OR', 'Floor'] = df_grouping_map['Care area/Department name']

In [3]:
df['loc_from_bed_mapped'] = df["loc_from"].replace(bed_map)
df['loc_dest_bed_mapped'] = df['loc_dest'].replace(bed_map)
#merge bed unit with pickup location 
df = df.merge(df_grouping_map, left_on = 'loc_from_bed_mapped', right_on = 'Care area/Department name', how = 'left')[['TransportID', 'LineNBR', 'EventInstantLocalDTS', 'AssignmentStatusCD',
       'AssignmentStatusDSC', 'loc_from', 'loc_dest', 'req_dept',
       'RequestDepartmentID', 'TransportFromPatientLocationFacilityID',
       'TransportToPatientLocationFacilityID', 'WorkerID', 'TransportModeDSC',
       'RequestSourceDSC', 'PriorityDSC', 'CurrentStatusDSC', 'SectorID',
       'loc_from_bed_mapped', 'loc_dest_bed_mapped', 'Floor']].rename(columns ={
    'Floor':'from_floor'
})

#merge bed unit with destination 
df = df.merge(df_grouping_map, left_on = 'loc_dest_bed_mapped', right_on = 'Care area/Department name', how = 'left')[['TransportID', 'LineNBR', 'EventInstantLocalDTS', 'AssignmentStatusCD',
       'AssignmentStatusDSC', 'loc_from', 'loc_dest', 'req_dept',
       'RequestDepartmentID', 'TransportFromPatientLocationFacilityID',
       'TransportToPatientLocationFacilityID', 'WorkerID', 'TransportModeDSC',
       'RequestSourceDSC', 'PriorityDSC', 'CurrentStatusDSC', 'SectorID',
       'loc_from_bed_mapped', 'loc_dest_bed_mapped', 'from_floor', 'Floor']].rename(columns ={
    'Floor':'dest_floor'
})

In [4]:
#add missing values in pickup location 
df.loc[df['loc_from_bed_mapped'].str.startswith('ED CDU'),'from_floor'] = 'Lunder 1'
df.loc[df['loc_from_bed_mapped'].str.lower().str.startswith('mgh blake 4'),'from_floor'] = 'Blake 4'
df.loc[df['loc_from_bed_mapped'].str.startswith('Blake 4'),'from_floor'] = 'Blake 4'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH Cath'),'from_floor'] = 'Blake 9'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH GRAY 4'),'from_floor'] = 'Gray Bigelow 4'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH Wang 3'),'from_floor'] = 'Wang 3'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH Lunder 2'),'from_floor'] = 'Lunder 2'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH Lunder 3'),'from_floor'] = 'Lunder 3'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH Lunder 4'),'from_floor'] = 'Lunder 4'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH Ellison 3'),'from_floor'] = 'Ellison 3'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH MEEI 11'),'from_floor'] = 'MEEI 11'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH Wang 3'),'from_floor'] = 'Wang 3'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH White 13'),'from_floor'] = 'White 13'
df.loc[df['loc_from_bed_mapped'].str.startswith('MGH White 3'),'from_floor'] = 'White 3'
df.loc[df['loc_from_bed_mapped'].str.startswith('B10'),'from_floor'] = 'Blake 10'
df.loc[df['loc_from_bed_mapped'].str.startswith('B14'),'from_floor'] = 'Blake 14'
df.loc[df['loc_from_bed_mapped'].str.startswith('E03'),'from_floor'] = 'Ellison 3'
df.loc[df['loc_from_bed_mapped'].str.startswith('E07'),'from_floor'] = 'Ellison 7'
df.loc[df['loc_from_bed_mapped'].str.startswith('E13'),'from_floor'] = 'Ellison 13'
df.loc[df['loc_from_bed_mapped'].str.startswith('E18'),'from_floor'] = 'Ellison 18'
df.loc[df['loc_from_bed_mapped'].str.startswith('W03'),'from_floor'] = 'White 3'
df.loc[df['loc_from_bed_mapped'].str.startswith('W12'),'from_floor'] = 'White 12'
df.loc[df['loc_from_bed_mapped'].str.endswith('YAW3'),'from_floor'] = 'Yawkey 3'

#add missing values in destination 
df.loc[df['loc_dest_bed_mapped'].str.startswith('ED CDU'),'dest_floor'] = 'Lunder 1'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH Blake 4'),'dest_floor'] = 'Blake 4'
df.loc[df['loc_dest_bed_mapped'].str.startswith('Blake 4'),'dest_floor'] = 'Blake 4'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH Cath'),'dest_floor'] = 'Blake 9'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH GRAY 4'),'dest_floor'] = 'Gray Bigelow 4'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH Wang 3'),'dest_floor'] = 'Wang 3'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH Lunder 2'),'dest_floor'] = 'Lunder 2'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH Lunder 3'),'dest_floor'] = 'Lunder 3'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH Lunder 4'),'dest_floor'] = 'Lunder 4'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH Ellison 3'),'dest_floor'] = 'Ellison 3'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH MEEI 11'),'dest_floor'] = 'MEEI 11'
df.loc[df['loc_dest_bed_mapped'].str.lower().str.startswith('mgh wang 3'),'dest_floor'] = 'Wang 3'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH White 13'),'dest_floor'] = 'White 13'
df.loc[df['loc_dest_bed_mapped'].str.startswith('MGH White 3'),'dest_floor'] = 'White 3'
df.loc[df['loc_dest_bed_mapped'].str.startswith('B10'),'dest_floor'] = 'Blake 10'
df.loc[df['loc_dest_bed_mapped'].str.startswith('B14'),'dest_floor'] = 'Blake 14'
df.loc[df['loc_dest_bed_mapped'].str.startswith('E03'),'dest_floor'] = 'Ellison 3'
df.loc[df['loc_dest_bed_mapped'].str.startswith('E07'),'dest_floor'] = 'Ellison 7'
df.loc[df['loc_dest_bed_mapped'].str.startswith('E13'),'dest_floor'] = 'Ellison 13'
df.loc[df['loc_dest_bed_mapped'].str.startswith('E18'),'dest_floor'] = 'Ellison 18'
df.loc[df['loc_dest_bed_mapped'].str.startswith('W03'),'dest_floor'] = 'White 3'
df.loc[df['loc_dest_bed_mapped'].str.startswith('W12'),'dest_floor'] = 'White 12'
df.loc[df['loc_dest_bed_mapped'].str.endswith('YAW3'),'dest_floor'] = 'Yawkey 3'

In [5]:
#replace missing in from_floor with loc_from and dest_floor and loc_dest
df.loc[df['from_floor'].isna(), 'from_floor'] = df['loc_from']
df.loc[df['dest_floor'].isna(), 'dest_floor'] = df['loc_dest']
#remove BC BOP POOL 
df = df[(df.from_floor != 'BC BOP POOL') | (df.dest_floor != 'BC BOP POOL')]
#remove self loop trip 
df = df[df['from_floor'] != df['dest_floor']]

### Find first timestamps

In [6]:
# Filter completed requests
df = df.loc[df.CurrentStatusDSC == 'Completed']

# Generate timestamps for each request
df_ts_pended = df.groupby(['TransportID']).apply(lambda x: x[x['AssignmentStatusDSC'] == 'Pending' ]['EventInstantLocalDTS'].min()).to_frame().reset_index().rename(columns = {0: 'first_pending_timestamp'})

df_ts_assigned = df.groupby(['TransportID']).apply(lambda x: x[x['AssignmentStatusDSC'] == 'Assigned']['EventInstantLocalDTS'].min()).to_frame().\
reset_index().rename(columns = {0:'first_assigned_timestamp'})

df_ts_acknowledged = df.groupby(['TransportID']).apply(lambda x: x[x['AssignmentStatusDSC'] == 'Acknowledged']['EventInstantLocalDTS'].min()).to_frame().\
reset_index().rename(columns = {0:'first_acknowledged_timestamp'})

df_ts_in_progress = df.groupby(['TransportID']).apply(lambda x: x[x['AssignmentStatusDSC'] == 'In Progress']['EventInstantLocalDTS'].min()).to_frame().\
reset_index().rename(columns = {0:'first_in_progress_timestamp'})

df_ts_completed = df.groupby(['TransportID']).apply(lambda x: x[x['AssignmentStatusDSC']=='Completed']['EventInstantLocalDTS'].max()).to_frame().\
reset_index().rename(columns = {0:'completed_timestamp'})


df_ts = df_ts_pended.merge(df_ts_assigned , on='TransportID').merge(df_ts_acknowledged , on='TransportID').merge(df_ts_in_progress , on='TransportID').merge(df_ts_completed , on='TransportID')


In [7]:
# Ensure pending, assigned, acknowledged, in progress is strictly ordered
# in order to calculate queue status 
from datetime import datetime, timedelta

def f(i, j, k): 
    if k == 1: 
        return max(i, j + timedelta(seconds=1))
    if k == 2:
        return max(i, j + timedelta(seconds=2))
    if k == 3: 
        return max(i, j + timedelta(seconds=3))
    if k == 4:
        return max(i, j + timedelta(seconds=4))
        
    
 
ass_ts= df_ts.apply(lambda x: f(x.first_assigned_timestamp, x.first_pending_timestamp, 1), axis=1)
ack_ts= df_ts.apply(lambda x: f(x.first_acknowledged_timestamp, x.first_pending_timestamp, 2), axis=1)
ip_ts = df_ts.apply(lambda x: f(x.first_in_progress_timestamp, x.first_pending_timestamp, 3), axis=1)
cp_ts = df_ts.apply(lambda x: f(x.completed_timestamp, x.first_pending_timestamp, 4), axis=1)

df_ts['first_assigned_timestamp'] = ass_ts
df_ts['first_acknowledged_timestamp']  = ack_ts
df_ts['first_in_progress_timestamp'] = ip_ts
df_ts['completed_timestamp'] = cp_ts
df_ts['first_pending_day_of_week'] = df_ts.first_pending_timestamp.dt.day_name()
df_ts['first_pending_hour_of_day'] = df_ts.first_pending_timestamp.astype(str).str[11:13]

#generate outcome labels 
df = df.merge(df_ts, on = 'TransportID')
df['wait_time_min'] = (df['first_in_progress_timestamp']-df['first_pending_timestamp'])/pd.Timedelta(minutes=1)
df['completion_time_min'] = (df['completed_timestamp']-df['first_in_progress_timestamp'])/pd.Timedelta(minutes=1)
df['total_time_min'] = df['wait_time_min'] + df['completion_time_min']

### Generate number of active workers 

In [8]:
# generate number of workers 
df.loc[:, 'EventInstantLocalDTS_hour'] = df['EventInstantLocalDTS'].dt.floor('h')
df_wrkr = df.loc[df.AssignmentStatusDSC.isin(['Completed', 'In Progress'])]
df_wrkr_cnts = df_wrkr.groupby('EventInstantLocalDTS_hour').WorkerID.nunique().reset_index().rename(columns = {'WorkerID': 'Num_Active_Wrkrs_Hourly'})
# df_wrkr_cnts - 2 columns: EventInstantLocalDTS_hour, Num_Active_Wrkrs_Hourly
df_first_pending_h =  df.groupby(['TransportID']).apply(lambda x: x[x['AssignmentStatusDSC'] == 'Pending' ]['EventInstantLocalDTS_hour'].min()).to_frame().reset_index().rename(columns = {0: 'first_pending_hour'})
# df_central_first_pending_h - 2 columns: TransportID, first_pending_hour
df_wrkr = df_first_pending_h.merge(df_wrkr_cnts, left_on = 'first_pending_hour', right_on = 'EventInstantLocalDTS_hour', how = 'left')[['TransportID', 'Num_Active_Wrkrs_Hourly']]
df_wrkr.Num_Active_Wrkrs_Hourly = df_wrkr.Num_Active_Wrkrs_Hourly.fillna(0)
# df_wrkr - 2 columns: 'TransportID', 'Num_Active_Wrkrs_Hourly'
df = df.merge(df_wrkr, on = 'TransportID', how = 'left')
df.Num_Active_Wrkrs_Hourly = df.Num_Active_Wrkrs_Hourly.fillna(0)


In [9]:
# Turn day/hour into numerical values
def day2num(row):
    x = str(row["first_pending_day_of_week"])
    if x == "Monday":
        return 1
    elif x == "Tuesday":
        return 2
    elif x == "Wednesday":
        return 3
    elif x == "Thursday":
        return 4
    elif x == "Friday":
        return 5
    elif x == "Saturday":
        return 6
    else:
        return 7


df["first_pending_day_of_week"] = df.apply(day2num, axis = 1)

# genearte Year variable
df["year"] = (df["first_pending_timestamp"].astype(str).str.slice(0,4)).astype(int)

### Central Data 

In [11]:
#get trip counts 
df_completed = df.loc[df.AssignmentStatusDSC == 'Completed']
df_central = df_completed.loc[~df_completed.SectorID.isin(["732",  "1065", "1066", "1143","1156"])]
df_central['trip_count'] = df_central.groupby(['from_floor', 'dest_floor']).completion_time_min.transform('size')
#get median as proxy for location distance 
df_central['completion_time_median'] = df_central.groupby(['from_floor', 'dest_floor']).completion_time_min.transform('median')
#only keep location pairs with > 200 counts (around 90 percentile of trip counts)
df_central = df_central[df_central['trip_count'] > 200]
#remove completion time > 34 mins (99.5 percentile)
df_central = df_central[df_central['completion_time_min'] <= 34]
#remove wait time > 90 (99.5 percentile)
df_central = df_central[df_central['wait_time_min'] <= 90]
#remove completion time = 0
df_central = df_central[df_central['completion_time_min'] != 0]
col = ['TransportID', 'TransportModeDSC','from_floor', 'dest_floor', 'first_pending_day_of_week', 'first_pending_hour_of_day','year', 'completion_time_median','Num_Active_Wrkrs_Hourly','wait_time_min', 'completion_time_min','total_time_min']
df_central = df_central[col].drop_duplicates()
#NA in TransportMode <1%, drop NA
df_central = df_central.dropna()
#convert mode to dummy variables 
df_central = pd.get_dummies(df_central, columns=["TransportModeDSC"], prefix=["mode_is"])

/var/folders/y4/m6pvx7sn327_q5sxknfvsb100000gn/T/ipykernel_64826/1388702745.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_central['trip_count'] = df_central.groupby(['from_floor', 'dest_floor']).completion_time_min.transform('size')
/var/folders/y4/m6pvx7sn327_q5sxknfvsb100000gn/T/ipykernel_64826/1388702745.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_central['completion_time_median'] = df_central.groupby(['from_floor', 'dest_floor']).completion_time_min.transform('median')


### Central data: Feature Generation for busy day, hour and classification label

In [12]:
# busyDay 1 or 5
# busyHour 6 or 13

def fun1(x): 
    return 1 if x == 1 or x == 5 else 0 
    
df_central["busyDay"] = df_central.apply(lambda row : fun1(row['first_pending_day_of_week']), axis = 1)


def fun2(x): 
    return 1 if x == 6 or x == 13 else 0 
    
    
df_central["busyHour"] = df_central.apply(lambda row : fun2(row['first_pending_hour_of_day']), axis = 1)



# generating labels 
df_central.groupby(['from_floor','dest_floor'])['total_time_min'].describe()
low_q = df_central.groupby(['from_floor','dest_floor'])['total_time_min'].quantile(q = 0.25)
low_q = pd.DataFrame(low_q)
low_q = low_q.reset_index()
high_q = df_central.groupby(['from_floor','dest_floor'])['total_time_min'].quantile(q = 0.75)
high_q = pd.DataFrame(high_q)
high_q = high_q.reset_index()


df_central = df_central.merge(low_q, left_on = ['from_floor', 'dest_floor'], 
                     right_on = ['from_floor', 'dest_floor'],
                    suffixes=("", "_25_pct"), how ='left')
df_central = df_central.merge(high_q, left_on = ['from_floor', 'dest_floor'], 
                     right_on = ['from_floor', 'dest_floor'],
                    suffixes=("", "_75_pct"), how = 'left')


def func(row):
    if row['total_time_min'] < row['total_time_min_25_pct']:
        return "green"
    elif row['total_time_min'] > row['total_time_min_75_pct']:
        return "red"
    else:
        return "yellow"


 
df_central['signal'] = df_central.apply(func, axis=1)




# Percentage breakdown
df_central['pending_pctg'] = df_central['pending']/df_central['queue_size']
df_central['assigned_pctg'] = df_central['assigned']/df_central['queue_size']
df_central['acknowledged_pctg'] = df_central['acknowledged']/df_central['queue_size']
df_central['inprogress_pctg'] = df_central['inprogress']/df_central['queue_size']
df_central.fillna(0, inplace = True)
'''
pending_per = df['pending'] / df['queue_size']
assigned_per = df['assigned'] / df['queue_size']
acknowledged_per = df['acknowledged'] / df['queue_size']
inprogress_per = df['inprogress'] / df['queue_size']


status = [pending_per, assigned_per, acknowledged_per, inprogress_per]
labels = ['Pending', 'Assigned', 'Acknowledged', 'In Progress']


'''


KeyError: 'pending'

In [13]:
#load ED queue size -- see R script 
ed_queue = pd.read_csv('ed_queue.csv')
ed_queue = ed_queue[['TransportID', 'queue_size', 'pending', 'assigned', 'acknowledged', 'inprogress']]

### Emergency Department Data 

In [14]:
df_ed = df_completed.loc[df_completed.SectorID.isin(["732",  "1065", "1066", "1143","1156"])]
df_ed['trip_count'] = df_ed.groupby(['from_floor', 'dest_floor']).completion_time_min.transform('size')
#get median as proxy for location distance 
df_ed['completion_time_median'] = df_ed.groupby(['from_floor', 'dest_floor']).completion_time_min.transform('median')

/var/folders/y4/m6pvx7sn327_q5sxknfvsb100000gn/T/ipykernel_64826/2467271708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ed['trip_count'] = df_ed.groupby(['from_floor', 'dest_floor']).completion_time_min.transform('size')
/var/folders/y4/m6pvx7sn327_q5sxknfvsb100000gn/T/ipykernel_64826/2467271708.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ed['completion_time_median'] = df_ed.groupby(['from_floor', 'dest_floor']).completion_time_min.transform('median')


In [15]:
#exclude all destinations that are ed/ed radiology -already filtered on sector ID 
df_ed = df_ed.loc[~df_ed.loc_dest.str.startswith('MGH ED')]
df_ed = df_ed.loc[~df_ed.loc_dest.isin(['MGH IMG CT ER MG WHT1', 'MGH IMG MR ER MG WHT1', 'MGH IMG XR ER MG WHT1',\
                                       'MGH RAD MRI ER WH1', 'MGH EMERGENCY BIGELOW 7', 'MGH EMERGENCY WHITE 13'])]

In [16]:
#only keep location pairs with > 200 counts (around 95-97 percentile)
df_ed = df_ed[df_ed['trip_count'] > 200]
#remove completion time > 28 mins (99.5 percentile)
df_ed = df_ed[df_ed['completion_time_min'] <= 28]
#remove wait time > 66 (99.5 percentile)
df_ed = df_ed[df_ed['wait_time_min'] <= 66]
#remove completion time = 0
df_ed = df_ed[df_ed['completion_time_min'] != 0]
df_ed = df_ed[col].drop_duplicates()
#NA in TransportMode <1%, drop NA
df_ed = df_ed.dropna()
#join queue size data 
df_ed = df_ed.merge(ed_queue, on = 'TransportID', how = 'left')
#convert mode to dummy variables 
df_ed = pd.get_dummies(df_ed, columns=["TransportModeDSC"], prefix=["mode_is"])

In [17]:
df_ed.isnull().sum()

TransportID                     0
from_floor                      0
dest_floor                      0
first_pending_day_of_week       0
first_pending_hour_of_day       0
year                            0
completion_time_median          0
Num_Active_Wrkrs_Hourly         0
wait_time_min                   0
completion_time_min             0
total_time_min                  0
queue_size                      0
pending                         0
assigned                        0
acknowledged                    0
inprogress                      0
mode_is_Bariatric Stretcher     0
mode_is_Bariatric Wheelchair    0
mode_is_Bed                     0
mode_is_Stretcher               0
mode_is_Walking                 0
mode_is_Wheelchair              0
mode_is_Wheelchair with O2      0
dtype: int64

### ED data: Feature Generation for busy day, hour and classification label

In [18]:
#nonbusy hour from 11 to 6 

def nonbusyhour(x): 
    return 1 if x == 23 or x == 0 or x == 1 or x == 2 or x == 3 or x == 4 or x == 5 or x == 6 else 0 
    
    
df_ed["nonbusyHour"] = df_ed.apply(lambda row : nonbusyhour(row['first_pending_hour_of_day']), axis = 1)



# generating labels 
df_ed.groupby(['from_floor','dest_floor'])['total_time_min'].describe()
low_q = df_ed.groupby(['from_floor','dest_floor'])['total_time_min'].quantile(q = 0.25)
low_q = pd.DataFrame(low_q)
low_q = low_q.reset_index()
high_q = df_ed.groupby(['from_floor','dest_floor'])['total_time_min'].quantile(q = 0.75)
high_q = pd.DataFrame(high_q)
high_q = high_q.reset_index()


df_ed = df_ed.merge(low_q, left_on = ['from_floor', 'dest_floor'], 
                     right_on = ['from_floor', 'dest_floor'],
                    suffixes=("", "_25_pct"), how ='left')
df_ed = df_ed.merge(high_q, left_on = ['from_floor', 'dest_floor'], 
                     right_on = ['from_floor', 'dest_floor'],
                    suffixes=("", "_75_pct"), how = 'left')


def classification_label(row):
    print(row['total_time_min'])
    if row['total_time_min'] < row['total_time_min_25_pct']:
        return "green"
    elif row['total_time_min'] > row['total_time_min_75_pct']:
        return "red"
    else:
        return "yellow"
df_ed['signal'] = df_ed.apply(classification_label, axis=1)




# Percentage breakdown
df_ed['pending_pctg'] = df_ed['pending']/df_ed['queue_size']
df_ed['assigned_pctg'] = df_ed['assigned']/df_ed['queue_size']
df_ed['acknowledged_pctg'] = df_ed['acknowledged']/df_ed['queue_size']
df_ed['inprogress_pctg'] = df_ed['inprogress']/df_ed['queue_size']
df_ed.fillna(0, inplace = True)
'''
pending_per = df['pending'] / df['queue_size']
assigned_per = df['assigned'] / df['queue_size']
acknowledged_per = df['acknowledged'] / df['queue_size']
inprogress_per = df['inprogress'] / df['queue_size']


status = [pending_per, assigned_per, acknowledged_per, inprogress_per]
labels = ['Pending', 'Assigned', 'Acknowledged', 'In Progress']


'''


5.0
34.0
10.0
8.0
4.0
20.0
24.0
9.0
20.0
41.0
11.0
8.0
8.0
37.0
34.0
6.0
5.0
11.0
6.0
24.0
31.0
35.0
14.0
4.0
38.0
4.0
7.0
14.0
32.0
3.0
5.0
8.0
7.0
4.0
27.0
9.0
15.0
11.0
15.0
13.0
7.0
11.0
9.0
69.0
9.0
5.0
17.0
9.0
8.0
9.0
16.0
28.0
12.0
16.0
8.0
8.0
6.0
11.0
5.0
11.0
9.0
8.0
4.0
8.0
22.0
8.0
7.0
13.0
21.0
24.0
50.0
35.0
29.0
30.0
35.0
15.0
36.0
6.0
11.0
14.0
29.0
25.0
27.0
12.0
58.0
36.0
16.0
6.0
13.0
16.0
16.0
21.0
14.0
10.0
6.0
26.0
37.0
25.0
26.0
27.0
25.0
21.0
20.0
16.0
22.0
20.0
14.0
11.0
8.0
6.0
45.0
16.0
7.0
5.0
9.0
11.0
12.0
6.0
10.0
5.0
12.0
46.0
17.0
9.0
4.0
7.0
13.0
23.0
9.0
5.0
9.0
6.0
20.0
26.0
5.0
23.0
36.0
11.0
11.0
13.0
15.0
5.0
4.0
40.0
9.0
4.0
5.0
13.0
11.0
13.0
6.0
13.0
7.0
11.0
3.0
8.0
10.0
17.0
23.0
6.0
15.0
22.0
43.0
65.0
5.0
8.0
6.0
23.0
37.0
5.0
16.0
5.0
40.0
8.0
41.0
46.0
2.0
13.0
56.0
50.0
11.0
12.0
16.0
20.0
13.0
12.0
14.0
32.0
22.0
6.0
30.0
11.0
15.0
20.0
32.0
10.0
39.0
14.0
27.0
11.0
12.0
18.0
21.0
8.0
17.0
11.0
19.0
22.0
14.0
7.0
7.0
9.0
11.0
9.0
6.0
13

12.0
12.0
26.0
6.0
7.0
6.0
5.0
6.0
25.0
9.0
4.0
6.0
6.0
21.0
6.0
5.0
8.0
4.0
4.0
6.0
54.0
3.0
4.0
5.0
6.0
22.0
3.0
6.0
4.0
6.0
5.0
10.0
11.0
5.0
13.0
7.0
6.0
5.0
19.0
33.0
4.0
7.0
6.0
2.0
7.0
6.0
15.0
14.0
9.0
6.0
6.0
12.0
4.0
6.0
11.0
43.0
40.0
8.0
35.0
38.0
45.0
11.0
34.0
8.0
43.0
8.0
39.0
11.0
25.0
42.0
11.0
15.0
51.0
15.0
10.0
43.0
12.0
38.0
5.0
28.0
18.0
30.0
10.0
29.0
23.0
30.0
5.0
6.0
12.0
13.0
11.0
29.0
19.0
14.0
10.0
6.0
35.0
17.0
8.0
10.0
17.0
14.0
12.0
24.0
14.0
9.0
30.0
23.0
12.0
19.0
22.0
23.0
9.0
16.0
16.0
22.0
35.0
40.0
37.0
9.0
50.0
13.0
23.0
23.0
7.0
12.0
13.0
28.0
11.0
13.0
30.0
27.0
37.0
33.0
10.0
24.0
21.0
12.0
18.0
7.0
11.0
8.0
13.0
8.0
33.0
38.0
12.0
6.0
7.0
8.0
8.0
8.0
14.0
52.0
4.0
7.0
7.0
22.0
14.0
9.0
28.0
8.0
12.0
35.0
13.0
6.0
11.0
15.0
12.0
12.0
9.0
18.0
9.0
7.0
6.0
24.0
4.0
3.0
16.0
17.0
3.0
29.0
7.0
38.0
16.0
65.0
27.0
6.0
5.0
11.0
12.0
14.0
41.0
18.0
17.0
9.0
8.0
10.0
9.0
6.0
6.0
7.0
3.0
16.0
12.0
3.0
29.0
26.0
38.0
7.0
48.0
6.0
5.0
13.0
43.0
8.0
14.0
9.

5.0
5.0
11.0
12.0
5.0
3.0
11.0
10.0
7.0
15.0
21.0
7.0
7.0
7.0
4.0
28.0
17.0
23.0
15.0
5.0
4.0
9.0
19.0
6.0
16.0
18.0
15.0
14.0
14.0
5.0
6.0
21.0
11.0
6.0
7.0
8.0
19.0
12.0
13.0
3.0
7.0
9.0
8.0
19.0
7.0
18.0
6.0
4.0
11.0
8.0
13.0
4.0
25.0
4.0
8.0
10.0
15.0
3.0
7.0
8.0
3.0
33.0
28.0
18.0
8.0
46.0
17.0
15.0
45.0
16.0
25.0
10.0
31.0
5.0
38.0
15.0
12.0
12.0
21.0
8.0
14.0
6.0
21.0
17.0
13.0
18.0
11.0
8.0
27.0
20.0
28.0
27.0
14.0
30.0
31.0
32.0
10.0
15.0
9.0
11.0
7.0
4.0
18.0
14.0
26.0
43.0
48.0
12.0
18.0
9.0
7.0
33.0
41.0
4.0
22.0
46.0
27.0
34.0
36.0
46.0
30.0
44.0
56.0
12.0
9.0
6.0
38.0
19.0
14.0
30.0
20.0
53.0
34.0
25.0
10.0
13.0
16.0
20.0
21.0
33.0
26.0
26.0
19.0
23.0
14.0
15.0
15.0
14.0
25.0
11.0
11.0
17.0
14.0
13.0
40.0
13.0
9.0
5.0
7.0
14.0
10.0
26.0
5.0
7.0
6.0
4.0
10.0
48.0
10.0
13.0
10.0
37.0
10.0
6.0
7.0
42.0
6.0
6.0
8.0
9.0
5.0
54.0
7.0
7.0
45.0
7.0
11.0
10.0
19.0
24.0
9.0
9.0
36.0
32.0
8.0
6.0
7.0
9.0
8.0
58.0
6.0
7.0
11.0
13.0
5.0
8.0
10.0
24.0
29.0
9.0
9.0
5.0
6.0
7.0
18.0
11.0

15.0
25.0
33.0
8.0
14.0
15.0
27.0
31.0
37.0
27.0
15.0
27.0
24.0
34.0
47.0
17.0
28.0
14.0
26.0
14.0
23.0
13.0
11.0
32.0
15.0
13.0
18.0
5.0
5.0
32.0
6.0
27.0
6.0
7.0
5.0
6.0
4.0
5.0
7.0
27.0
8.0
5.0
4.0
34.0
21.0
3.0
8.0
5.0
5.0
7.0
11.0
13.0
10.0
11.0
6.0
10.0
7.0
4.0
3.0
10.0
3.0
9.0
6.0
29.0
11.0
34.0
4.0
9.0
16.0
23.0
9.0
14.0
5.0
6.0
16.0
42.0
10.0
7.0
3.0
68.0
68.0
15.0
59.0
1.0
41.0
4.0
49.0
4.0
8.0
27.0
11.0
5.0
4.0
32.0
4.0
25.0
9.0
9.0
5.0
24.0
16.0
11.0
13.0
24.0
18.0
33.0
17.0
11.0
15.0
32.0
15.0
14.0
22.0
13.0
16.0
9.0
12.0
21.0
17.0
6.0
17.0
19.0
17.0
31.0
43.0
29.0
52.0
34.0
6.0
30.0
18.0
49.0
24.0
37.0
24.0
8.0
24.0
20.0
29.0
16.0
15.0
25.0
34.0
33.0
5.0
43.0
32.0
9.0
6.0
5.0
40.0
10.0
25.0
25.0
24.0
30.0
45.0
58.0
18.0
18.0
66.0
9.0
14.0
18.0
10.0
8.0
36.0
44.0
56.0
27.0
8.0
1.0
45.0
8.0
16.0
53.0
10.0
42.0
7.0
26.0
30.0
15.0
7.0
38.0
42.0
42.0
10.0
52.0
7.0
9.0
66.0
8.0
7.0
68.0
6.0
8.0
77.0
8.0
5.0
6.0
25.0
49.0
6.0
5.0
54.0
8.0
12.0
13.0
17.0
14.0
35.0
21.0
25.0
15.0


21.0
10.0
7.0
15.0
12.0
8.0
7.0
10.0
21.0
27.0
4.0
13.0
7.0
41.0
7.0
9.0
6.0
9.0
21.0
8.0
17.0
4.0
7.0
10.0
24.0
37.0
5.0
10.0
16.0
12.0
20.0
21.0
9.0
13.0
48.0
7.0
24.0
7.0
9.0
20.0
27.0
6.0
13.0
3.0
26.0
32.0
11.0
8.0
58.0
8.0
52.0
56.0
28.0
6.0
60.0
9.0
8.0
19.0
12.0
17.0
7.0
45.0
9.0
18.0
10.0
28.0
12.0
37.0
49.0
6.0
6.0
20.0
6.0
4.0
7.0
3.0
6.0
6.0
3.0
5.0
3.0
6.0
6.0
11.0
15.0
6.0
5.0
6.0
3.0
20.0
9.0
5.0
4.0
5.0
13.0
11.0
10.0
7.0
8.0
9.0
14.0
4.0
13.0
5.0
7.0
16.0
5.0
9.0
6.0
7.0
5.0
14.0
6.0
9.0
19.0
5.0
18.0
10.0
18.0
12.0
12.0
25.0
8.0
4.0
10.0
28.0
17.0
29.0
27.0
35.0
54.0
51.0
30.0
29.0
18.0
28.0
22.0
34.0
28.0
47.0
27.0
55.0
10.0
18.0
14.0
9.0
4.0
16.0
15.0
12.0
9.0
26.0
19.0
28.0
23.0
30.0
37.0
40.0
31.0
10.0
56.0
24.0
24.0
39.0
16.0
13.0
27.0
5.0
9.0
5.0
29.0
13.0
51.0
22.0
18.0
49.0
34.0
28.0
23.0
18.0
16.0
24.0
21.0
11.0
27.0
13.0
19.0
19.0
17.0
15.0
14.0
16.0
15.0
16.0
44.0
10.0
15.0
54.0
11.0
15.0
20.0
19.0
10.0
9.0
10.0
5.0
9.0
4.0
6.0
8.0
5.0
41.0
9.0
4.0
4.0
2.0


...

16.0
21.0
6.0
23.0
9.0
12.0
9.0
18.0
10.0
29.0
21.0
9.0
11.0
31.0
16.0
8.0
20.0
17.0
37.0
13.0
22.0
13.0
11.0
41.0
21.0
11.0
8.0
17.0
10.0
11.0
13.0
7.0
23.0
15.0
31.0
14.0
14.0
5.0
5.0
19.0
12.0
16.0
22.0
6.0
20.0
11.0
11.0
34.0
6.0
19.0
9.0
16.0
10.0
39.0
17.0
26.0
13.0
4.0
13.0
11.0
19.0
20.0
14.0
17.0
15.0
26.0
17.0
11.0
11.0
14.0
42.0
48.0
34.0
29.0
38.0
19.0
37.0
42.0
20.0
19.0
40.0
30.0
14.0
11.0
21.0
39.0
18.0
22.0
18.0
32.0
19.0
45.0
22.0
33.0
30.0
15.0
7.0
18.0
15.0
12.0
20.0
21.0
26.0
28.0
51.0
23.0
9.0
9.0
12.0
16.0
8.0
7.0
39.0
9.0
10.0
53.0
13.0
11.0
5.0
21.0
4.0
29.0
23.0
9.0
23.0
4.0
6.0
18.0
10.0
10.0
26.0
17.0
25.0
11.0
23.0
19.0
7.0
10.0
12.0
4.0
14.0
7.0
5.0
26.0
11.0
5.0
29.0
7.0
6.0
6.0
15.0
23.0
13.0
19.0
23.0
15.0
18.0
11.0
23.0
26.0
26.0
8.0
23.0
9.0
17.0
4.0
13.0
22.0
29.0
23.0
27.0
12.0
21.0
27.0
11.0
14.0
31.0
52.0
50.0
9.0
15.0
13.0
11.0
7.0
20.0
15.0
8.0
6.0
20.0
23.0
45.0
30.0
11.0
12.0
24.0
21.0
29.0
18.0
14.0
26.0
12.0
17.0
23.0
30.0
26.0
30.0
25.0
31.0

14.0
31.0
5.0
13.0
13.0
15.0
35.0
14.0
15.0
22.0
29.0
13.0
32.0
14.0
35.0
19.0
23.0
24.0
21.0
27.0
23.0
37.0
19.0
33.0
10.0
16.0
21.0
26.0
18.0
9.0
5.0
15.0
14.0
23.0
20.0
22.0
11.0
15.0
16.0
16.0
18.0
5.0
38.0
21.0
5.0
10.0
11.0
21.0
19.0
5.0
8.0
34.0
14.0
5.0
9.0
7.0
7.0
7.0
24.0
5.0
8.0
8.0
17.0
8.0
19.0
17.0
12.0
14.0
10.0
11.0
18.0
56.0
28.0
17.0
11.0
68.0
7.0
6.0
9.0
12.0
13.0
17.0
8.0
8.0
9.0
8.0
5.0
32.0
7.0
39.0
42.0
36.0
37.0
4.0
40.0
15.0
5.0
3.0
25.0
36.0
11.0
21.0
21.0
8.0
18.0
52.0
28.0
24.0
29.0
18.0
25.0
11.0
13.0
4.0
7.0
9.0
27.0
16.0
44.0
10.0
13.0
34.0
8.0
28.0
3.0
29.0
9.0
33.0
7.0
41.0
8.0
8.0
20.0
31.0
38.0
34.0
47.0
47.0
28.0
40.0
5.0
29.0
8.0
10.0
6.0
14.0
5.0
10.0
27.0
22.0
16.0
4.0
8.0
27.0
18.0
18.0
16.0
19.0
39.0
13.0
13.0
17.0
10.0
15.0
21.0
9.0
14.0
19.0
5.0
16.0
5.0
14.0
10.0
12.0
14.0
19.0
5.0
18.0
8.0
9.0
23.0
9.0
3.0
7.0
3.0
13.0
11.0
16.0
15.0
18.0
25.0
29.0
15.0
12.0
20.0
35.0
22.0
13.0
42.0
34.0
54.0
35.0
14.0
40.0
10.0
22.0
15.0
32.0
23.0
28.0
18.0

18.0
14.0
19.0
27.0
12.0
19.0
18.0
22.0
14.0
39.0
15.0
21.0
16.0
28.0
12.0
18.0
11.0
7.0
9.0
25.0
19.0
8.0
8.0
26.0
8.0
5.0
16.0
5.0
11.0
26.0
19.0
4.0
7.0
10.0
12.0
10.0
9.0
14.0
6.0
5.0
4.0
7.0
7.0
4.0
15.0
7.0
5.0
5.0
6.0
5.0
7.0
8.0
8.0
13.0
8.0
14.0
7.0
7.0
26.0
9.0
6.0
37.0
5.0
13.0
15.0
7.0
6.0
5.0
4.0
15.0
14.0
31.0
27.0
20.0
15.0
32.0
4.0
31.0
36.0
8.0
44.0
6.0
13.0
39.0
60.0
57.0
15.0
57.0
57.0
82.0
13.0
4.0
6.0
19.0
27.0
10.0
33.0
14.0
18.0
29.0
8.0
52.0
11.0
7.0
22.0
5.0
40.0
48.0
33.0
7.0
27.0
14.0
10.0
26.0
44.0
25.0
8.0
11.0
12.0
10.0
18.0
24.0
7.0
13.0
17.0
8.0
15.0
22.0
13.0
15.0
12.0
15.0
23.0
15.0
24.0
33.0
7.0
11.0
10.0
39.0
6.0
10.0
7.0
11.0
28.0
14.0
14.0
8.0
13.0
9.0
12.0
8.0
14.0
8.0
18.0
6.0
10.0
11.0
13.0
19.0
29.0
28.0
35.0
18.0
21.0
29.0
24.0
25.0
24.0
25.0
6.0
25.0
20.0
23.0
6.0
17.0
29.0
22.0
18.0
3.0
25.0
34.0
26.0
20.0
13.0
30.0
20.0
13.0
7.0
26.0
8.0
19.0
9.0
19.0
10.0
6.0
29.0
8.0
4.0
28.0
11.0
8.0
8.0
7.0
10.0
12.0
7.0
19.0
7.0
6.0
8.0
18.0
15.0
5.0
1

8.0
6.0
29.0
32.0
13.0
36.0
13.0
17.0
37.0
19.0
19.0
12.0
12.0
9.0
31.0
19.0
19.0
20.0
29.0
5.0
19.0
2.0
17.0
8.0
15.0
20.0
11.0
16.0
17.0
23.0
19.0
20.0
21.0
21.0
12.0
12.0
23.0
24.0
17.0
24.0
13.0
19.0
9.0
17.0
23.0
12.0
25.0
21.0
19.0
18.0
7.0
24.0
28.0
19.0
12.0
7.0
11.0
16.0
23.0
18.0
24.0
16.0
10.0
25.0
10.0
38.0
26.0
33.0
10.0
27.0
22.0
25.0
19.0
12.0
25.0
35.0
32.0
21.0
17.0
27.0
26.0
21.0
20.0
16.0
27.0
27.0
16.0
31.0
16.0
21.0
23.0
22.0
17.0
20.0
13.0
11.0
25.0
22.0
19.0
19.0
27.0
14.0
12.0
14.0
15.0
21.0
20.0
32.0
22.0
22.0
17.0
11.0
14.0
30.0
6.0
24.0
41.0
19.0
22.0
8.0
31.0
6.0
40.0
12.0
42.0
18.0
26.0
26.0
8.0
5.0
11.0
6.0
47.0
23.0
32.0
17.0
26.0
37.0
27.0
58.0
26.0
12.0
13.0
14.0
6.0
29.0
31.0
33.0
45.0
41.0
28.0
14.0
4.0
8.0
78.0
10.0
14.0
23.0
29.0
20.0
21.0
6.0
15.0
9.0
20.0
37.0
40.0
20.0
23.0
18.0
9.0
55.0
49.0
15.0
19.0
17.0
29.0
23.0
42.0
21.0
19.0
41.0
16.0
45.0
7.0
12.0
29.0
22.0
8.0
32.0
16.0
10.0
25.0
31.0
12.0
46.0
14.0
4.0
15.0
19.0
11.0
6.0
23.0
17.0
10.0


"\npending_per = df['pending'] / df['queue_size']\nassigned_per = df['assigned'] / df['queue_size']\nacknowledged_per = df['acknowledged'] / df['queue_size']\ninprogress_per = df['inprogress'] / df['queue_size']\n\n\nstatus = [pending_per, assigned_per, acknowledged_per, inprogress_per]\nlabels = ['Pending', 'Assigned', 'Acknowledged', 'In Progress']\n\n\n"

In [ ]:
df_ed.head()

In [ ]:
df_ed.columns

In [ ]:
df_ed.to_csv('df_ed_modeling.csv', index=False)

In [ ]:
df_ed.shape